# Original version

This is the first version ... using ugly OpenCV font.

See below for a slicker version.

In [1]:
import cv2
from keras.models import load_model
import numpy as np
from random import randrange
import time
from PIL import ImageFont, ImageDraw, Image

model = load_model('keras_model.h5', compile=False)
cap = cv2.VideoCapture(2)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
font = cv2.FONT_HERSHEY_SIMPLEX
font_pil = ImageFont.truetype("MochiyPopPOne-Regular.ttf", 80)
choices = ['rock','paper','scissors','neutral']
neutral_index = 3
scoreboard = {'human':0, 'computer':0}
win_target = 3

display_text = ["Ready?","","",""]
counting_down = True
capturing = False
playing = True
winner = False
start_time = time.time()
countdown = 3

while True:
    ret, frame = cap.read()
    if frame is None:
        print('no image captured')
        continue
    # crop the frame so it matches (approx) the crop from the teachable machine website
    frame = frame[100:750, 400:880]
    # countdown before capturing
    if counting_down:
        if time.time() - start_time > 1:
            if countdown == 0:
                capturing = True
                counting_down = False
                display_text[0] = "Pose now!"
                display_text[1] = ""
                computer_choice = randrange(0,3)
            else:
                display_text[1] = "{}... ".format(countdown)
                start_time = time.time()
                countdown -= 1
    elif capturing:
        resized_frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
        image_np = np.array(resized_frame)
        normalized_image = (image_np.astype(np.float32) / 127.0) - 1
        data[0] = normalized_image
        prediction = model.predict(data)
        max_val = None
        max_index = None
        for index,val in enumerate(prediction[0]):
            if (max_val is None or val > max_val):
                max_index = index
                max_val = val
        if max_index != neutral_index:
            matched_image = (choices[max_index])
            display_text[0] = f"You chose {matched_image}"
            capturing = False
    elif playing:
        display_text[1] = f"I chose {choices[computer_choice]}"
        if computer_choice == (max_index + 1) % 3:
            # computer won
            display_text[2] = "I win!"
            scoreboard['computer'] += 1
        elif max_index == (computer_choice + 1) % 3:
            # player won
            display_text[2] = "You win!"
            scoreboard['human'] += 1
        else:
            display_text[2] = "It's a draw"
            display_text[1] = display_text[1] + " too"
        display_text[3] = 'Press r for the next round'
        playing = False
    # show winner if someone has won already
    if (scoreboard["human"] == win_target or scoreboard["computer"] == win_target):
        win_text = ("I" if scoreboard["computer"] == win_target else "You") + " won!!"
        cv2.putText(frame,win_text,(30, 250), font, 2,(255,50,50),5,cv2.LINE_AA)
        cv2.putText(frame,"3-{}".format(min(scoreboard["human"],scoreboard["computer"])),(30, 310), font, 2,(255,50,50),5,cv2.LINE_AA)
        winner = True
        # remove the 'press r for next round' prompt
        display_text[3] = ""
    else:
        # show scoreboard at the bottom if no winner yet
        # background
        cv2.rectangle(frame, (20,475),(180,515), (255,255,255), -1)
        scoreboard_text = 'you: {0} | me: {1}'.format(scoreboard["human"],scoreboard["computer"])
        cv2.putText(frame,scoreboard_text, (30,500), font, 0.5, (20,20,50),1,cv2.LINE_AA)
    # display whatever messages are defined in display_text
    line_num = 0
    for text_line in display_text:
        cv2.putText(frame,text_line,(30, 75 + line_num * 40), font, 0.5 if line_num==3 else 1, (255,255,0) if line_num==2 else (255,255,255), 1 if line_num==3 else 2, cv2.LINE_AA)
        line_num += 1
    cv2.imshow('frame', frame)
    # Press q to close the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    if cv2.waitKey(1) & 0xFF == ord('r') and not winner:
        display_text = ["Ready?","","",""]
        capturing = False
        start_time = time.time()
        counting_down = True
        playing = True
        countdown = 3

# After the loop release the cap object
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()


2022-01-27 19:53:37.515130: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-27 19:53:37.515400: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-01-27 19:53:45.915306: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-27 19:53:46.079575: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


## Pillow (truetype) version

Nicer fonts. And a few other improvements.

In [34]:
import cv2
from keras.models import load_model
import numpy as np
from random import randrange
import time
from PIL import ImageFont, ImageDraw, Image

model = load_model('keras_model.h5', compile=False)
cap = cv2.VideoCapture(2)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
font_pil = ImageFont.truetype("MochiyPopPOne-Regular.ttf", 24)
font_big = ImageFont.truetype("MochiyPopPOne-Regular.ttf", 40)
font_huge = ImageFont.truetype("MochiyPopPOne-Regular.ttf", 70)
choices = ['rock','paper','scissors','neutral']
neutral_index = 3
scoreboard = {'human':0, 'computer':0}
win_target = 3

display_text = ["Ready?","","",""]
counting_down = True
capturing = False
playing = True
winner = False
start_time = time.time()
countdown = 3
countdown_speed = 1

while True:
    ret, frame = cap.read()
    if frame is None:
        print('no image captured')
        continue
    # crop the frame so it matches (approx) the crop from the teachable machine website
    frame = frame[100:750, 400:880]
    # countdown before capturing
    if counting_down:
        if time.time() - start_time > countdown_speed:
            if countdown == 0:
                capturing = True
                counting_down = False
                display_text[0] = "Pose now!"
                display_text[1] = ""
                computer_choice = randrange(0,3)
            else:
                display_text[1] = "{}... ".format(countdown)
                start_time = time.time()
                countdown -= 1
    elif capturing:
        resized_frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
        image_np = np.array(resized_frame)
        normalized_image = (image_np.astype(np.float32) / 127.0) - 1
        data[0] = normalized_image
        prediction = model.predict(data)
        max_val = None
        max_index = None
        for index,val in enumerate(prediction[0]):
            if (max_val is None or val > max_val):
                max_index = index
                max_val = val
        if max_index != neutral_index:
            matched_image = (choices[max_index])
            display_text[0] = f"You chose {matched_image}"
            capturing = False
    elif playing:
        display_text[1] = f"I chose {choices[computer_choice]}"
        if computer_choice == (max_index + 1) % 3:
            # computer won
            display_text[2] = "I win!"
            scoreboard['computer'] += 1
        elif max_index == (computer_choice + 1) % 3:
            # player won
            display_text[2] = "You win!"
            scoreboard['human'] += 1
        else:
            display_text[2] = "It's a draw"
            display_text[1] = display_text[1] + " too"
        display_text[3] = 'Press r for the next round'
        playing = False
    # fonts section
    # convert image to RGB from BGR
    cv2_im_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_im = Image.fromarray(cv2_im_rgb)
    draw = ImageDraw.Draw(pil_im)
    center_x = frame.shape[1]/2
    # show winner if someone has won already
    if (scoreboard["human"] == win_target or scoreboard["computer"] == win_target):
        win_text = ("I" if scoreboard["computer"] == win_target else "You") + " won\nthe game!!"
        draw.text((center_x,300), win_text, font=font_big, fill='red', align='center', anchor='mm')
        draw.text((center_x,425), "3-{}".format(min(scoreboard["human"],scoreboard["computer"])), font=font_huge, fill='orange', align='center', anchor='mm')
        winner = True
        # remove the 'press r for next round' prompt
        display_text[3] = ""
    else:
        # show scoreboard at the bottom if no winner yet
        # background
        draw.rounded_rectangle([(center_x/4,475),(center_x*7/4,530)], radius=10, outline='black', fill="white", width=2)
        scoreboard_text = 'you: {0} | me: {1}'.format(scoreboard["human"],scoreboard["computer"])
        # draw.text((center_x,500), scoreboard_text, font=font_pil, fill='green', anchor='mm')
        draw.text((center_x*5/8,500), 'you: {}'.format(scoreboard["human"]), font=font_pil, fill='green',anchor='mm')
        draw.text((center_x*11/8,500), 'me: {}'.format(scoreboard["computer"]), font=font_pil, fill='green',anchor='mm')
    # display whatever messages are defined in display_text
    line_num = 0
    for text_line in display_text:
        big_line = (line_num==2 or capturing or counting_down)
        draw.text((center_x, 75 + line_num * 48), text_line, font=font_big if big_line else font_pil, fill='blue' if line_num==2 else 'white', anchor='mm')
        line_num += 1
    frame = cv2.cvtColor(np.array(pil_im), cv2.COLOR_RGB2BGR)
    cv2.imshow('frame', frame)
    # Press q to close the window
    if (cv2.waitKey(1) & 0xFF == ord('q')) or winner:
        break
    if cv2.waitKey(1) & 0xFF == ord('r') and not winner:
        display_text = ["Ready?","","",""]
        capturing = False
        start_time = time.time()
        counting_down = True
        playing = True
        countdown = 3

# After the loop release the cap object
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()


2022-01-27 21:50:59.852603: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
